In [43]:
import requests    # 連線與抓資料所需要使用的套件

import bs4         # 剖析抓下來的網頁資料所需要的套件

import re

In [4]:
def connect_urls(url):
    
    try:
    
        response = requests.get(url, timeout = 3)  # 要求建立各種 HTTP 請求，從網頁伺服器上取得想要的資料，最多嘗試3次
        
        # Consider any status other than 2xx an error
        # 檢查狀態碼是否 OK，如果顯示 200 就代表沒問題
        if not response.status_code == 200:
            
            return "Error: Unexpected response {}".format(response), 0

        return response, 1
    
    except requests.exceptions.RequestException as e:
        
        # A serious problem happened, like an SSLError or InvalidURL
        return "Error: {}".format(e)

In [5]:
url1 = 'https://www.taiwanlottery.com.tw/'

# 如上面網址不作用請換以下的網址
url2 = 'https://www.taiwanlottery.com.tw/index_new.aspx'

test_urls = [url1, url2]


counter = 0    # 計算已經嘗試連線幾個不同的 URL 位址

max_connection_tries = len(test_urls)  # 連線不同 URL 位址的個數上限

for url in test_urls:
    
    print("\n連線網頁位址：'{}'".format(url))
    
    print("\n網頁下載中 ...")
    
    res = connect_urls(url)

    counter += 1
    
    if (len(res) == 2) and (res[1] == 1):
        
        html = res[0]
        
        print("\n網頁下載完成")
        
        break
    
    elif counter == max_connection_tries:
                    
        print("\n錯誤：網頁下載失敗！ ", res[0])
        
        raise   # 強制停止程式執行 (manually halt execution)


連線網頁位址：'https://www.taiwanlottery.com.tw/'

網頁下載中 ...

網頁下載完成


In [42]:
objSoup = bs4.BeautifulSoup( html.text, 'lxml' )    # 建立 BeautifulSoup 物件

dataTag = objSoup.find_all('div', {'id':'rightdown'})  

pattern = '<!--\*{14,15}([^\*]+)\*?\*{14}-->'

for i in range(len(dataTag)):                       # 列出含 contents_box02 的串列                 

    lottery_title = re.findall(pattern,str(dataTag[i]))
    lottery_time = dataTag[i].find_all('span', {'class':'font_black15'})

lottery_date = []
lottery_period = []

for i in range(len(lottery_time)):  
    
    date,period= str(lottery_time[i].text).split()
    
    lottery_date.append(date)
    
    lottery_period.append(period)

lottery_title.remove("大福彩區塊")

dict = {"Name": lottery_title,  
        "Date": lottery_date,
        "Period": lottery_period
        }

lottery_table = pd.DataFrame(dict,index=list(range(1,len(lottery_title)+1)))

In [7]:
lotter_all = []

contents_box01 = objSoup.select('.contents_box01') 

lotter_bingo = contents_box01[0].find_all('div', {'class':'ball_tx ball_yellow'})

lotter_bingo2 = contents_box01[0].find_all('div', {'class':'ball_red'})

lotter_bingo3 = contents_box01[0].find_all('div', {'class':'ball_blue_BB1'})

lotter_bingo4 = contents_box01[0].find_all('div', {'class':'ball_blue_BB2'})
                                            
lotter_all.append(lotter_bingo)

lotter_all[0].append(lotter_bingo2[0])

lotter_all[0].append(lotter_bingo3[0])

lotter_all[0].append(lotter_bingo4[0])

In [8]:
contents_box06 = objSoup.select('.contents_box06') 

lotter_winwin = contents_box06[0].find_all('div', {'class':'ball_tx ball_blue'})
                                            
lotter_all.append(lotter_winwin)

In [9]:
contents_box02 = objSoup.select('.contents_box02') 

lotter_power = contents_box02[0].find_all('div', {'class':'ball_tx ball_green'})

lotter_power2 = contents_box02[0].find_all('div', {'class':'ball_red'})

lotter_happy38 = contents_box02[1].find_all('div', {'class':'ball_tx ball_green'})

lotter_bighappy = contents_box02[2].find_all('div', {'class':'ball_tx ball_yellow'})

lotter_bighappy2 = contents_box02[2].find_all('div', {'class':'ball_red'})

lotter_happy49 = contents_box02[3].find_all('div', {'class':'ball_tx ball_yellow'})
                                            
lotter_all.append(lotter_power)

lotter_all[2].append(lotter_power2[0])

lotter_all.append(lotter_happy38)

lotter_all.append(lotter_bighappy)

lotter_all[4].append(lotter_bighappy2[0])

lotter_all.append(lotter_happy49)

In [10]:
contents_box03 = objSoup.select('.contents_box03') 

lotter_today539 = contents_box03[0].find_all('div', {'class':'ball_tx ball_lemon'})

lotter_happy39 = contents_box03[1].find_all('div', {'class':'ball_tx ball_lemon'})
                                            
lotter_all.append(lotter_today539)

lotter_all.append(lotter_happy39)

In [11]:
contents_box04 = objSoup.select('.contents_box04') 

lotter_star3 = contents_box04[0].find_all('div', {'class':'ball_tx ball_purple'})

lotter_star4 = contents_box04[1].find_all('div', {'class':'ball_tx ball_purple'})
                                            
lotter_all.append(lotter_star3)

lotter_all.append(lotter_star4)

In [12]:
def check_index(index):
    
    lotter_num = []
    
    num = []
    
    if(index == '1'):
        
        lotter_index = ['遊戲名稱','開出獎號','開出獎號','開出獎號', '超級獎號', '猜大小', '猜單雙']
        
        lotter_num.append(lottery_title[int(index)-1])
        
        for i in range(0,9):
        
            num.append(lotter_all[int(index)-1][i].text)
        
        lotter_num.append(num)
        
        num = []
        
        for i in range(9,18):
        
            num.append(lotter_all[int(index)-1][i].text)
        
        lotter_num.append(num)
        
        num = []
        
        for i in range(18,21):
        
            num.append(lotter_all[int(index)-1][i].text)
        
        lotter_num.append(num)
        
        for i in range(20,23):
        
            lotter_num.append(lotter_all[int(index)-1][i].text)
        
        lotter_Series = pd.Series(lotter_num, index = lotter_index, name="數字") 
        
        return lotter_Series.to_frame()

    if(index == '2'):
        
        lotter_index = ['遊戲名稱','開出順序','開出順序','大小順序', '大小順序']
        
        lotter_num.append(lottery_title[int(index)-1])
        
        for i in range(0,9):
        
            num.append(lotter_all[int(index)-1][i].text)
        
        lotter_num.append(num)
        
        num = []
        
        for i in range(9,12):
        
            num.append(lotter_all[int(index)-1][i].text)
        
        lotter_num.append(num)
        
        num = []
        
        for i in range(12,21):
        
            num.append(lotter_all[int(index)-1][i].text)
        
        lotter_num.append(num)
        
        num = []
        
        for i in range(21,24):
            
            num.append(lotter_all[int(index)-1][i].text)
        
        lotter_num.append(num)
        
        lotter_Series = pd.Series(lotter_num, index = lotter_index, name="數字") 
        
        return lotter_Series.to_frame()
    
    if(index == '3' or index == '4'or index == '5'or index == '6'):
        
        if(index == '3'):
            lotter_index = ['遊戲名稱','開出順序','大小順序', '第二區']
        elif(index == '5'):
            lotter_index = ['遊戲名稱','開出順序','大小順序', '特別號']
        else:
            lotter_index = ['遊戲名稱','開出順序','大小順序']
        
        
        lotter_num.append(lottery_title[int(index)-1])
        
        for i in range(0,6):
        
            num.append(lotter_all[int(index)-1][i].text)
        
        lotter_num.append(num)
        
        num = []
        
        for i in range(6,12):
        
            num.append(lotter_all[int(index)-1][i].text)
        
        lotter_num.append(num)
        
        num = []
        
        if(index == '3' or index == '5'):
            for i in range(12,13):
        
                lotter_num.append(lotter_all[int(index)-1][i].text)
                      
        lotter_Series = pd.Series(lotter_num, index = lotter_index, name="數字") 
        
        return lotter_Series.to_frame()
    
    if(index == '7' or index == '8'):
        
        lotter_index = ['遊戲名稱','開出順序','大小順序']
        
        lotter_num.append(lottery_title[int(index)-1])
        
        for i in range(0,5):
        
            num.append(lotter_all[int(index)-1][i].text)
        
        lotter_num.append(num)
        
        num = []
        
        for i in range(5,10):
        
            num.append(lotter_all[int(index)-1][i].text)
        
        lotter_num.append(num)
                      
        lotter_Series = pd.Series(lotter_num, index = lotter_index, name="數字") 
        
        return lotter_Series.to_frame()

    if(index == '9' or index == '10'):
        
        lotter_index = ['遊戲名稱','中獎號碼']
        
        lotter_num.append(lottery_title[int(index)-1])
        
        if(index == '9'):
        
            for i in range(0,3):
        
                num.append(lotter_all[int(index)-1][i].text)
        
            lotter_num.append(num)
        else:
            
            for i in range(0,4):
        
                num.append(lotter_all[int(index)-1][i].text)
        
            lotter_num.append(num)
            
        lotter_Series = pd.Series(lotter_num, index = lotter_index, name="數字") 
        
        return lotter_Series.to_frame()

In [13]:
lottery_table

,Name,Date,Period
1,BINGO BINGO,109/6/24,第109035580期
2,雙贏彩區塊,109/6/23,第109000150期
3,威力彩區塊,109/6/22,第109000050期
4,38樂合彩區塊,109/6/22,第109000050期
5,大樂透區塊,109/6/23,第109000058期
6,49樂合彩區塊,109/6/23,第109000058期
7,今彩539區塊,109/6/23,第109000150期
8,39樂合彩區塊,109/6/23,第109000150期
9,3星彩區塊,109/6/23,第109000150期
10,4星彩區塊,109/6/23,第109000150期


In [15]:
index = input("請輸入想看的獎券索引值:")

finall = check_index(index)

finall

請輸入想看的獎券索引值:2


,數字
遊戲名稱,雙贏彩區塊
開出順序,"[05 , 02 , 18 , 06 , 23 , 16 , 20 , 15 , 10 ]"
開出順序,"[03 , 21 , 04 ]"
大小順序,"[02 , 03 , 04 , 05 , 06 , 10 , 15 , 16 , 18 ]"
大小順序,"[20 , 21 , 23 ]"
